In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

import pandas as pd

from sklearn import linear_model

import matplotlib.pyplot as plt

%matplotlib inline
import datetime as dt
import numpy as np
import seaborn as sns
import pandas as pd

#Snowflake connection info is saved in config.py
from config import snowflake_conn_prop


# lets import some tranformations functions
from snowflake.snowpark.functions import udf, col, lit, translate, is_null, iff

In [2]:
from snowflake.snowpark import version
print(version.VERSION)
#session.close()
session = Session.builder.configs(snowflake_conn_prop).create()
session.sql("use database {}".format(snowflake_conn_prop['database'])).collect()
session.sql("create schema if not exists {}".format(snowflake_conn_prop['schema'])).collect()
session.sql("use schema {}".format(snowflake_conn_prop['schema'])).collect()
session.sql("use warehouse {}".format(snowflake_conn_prop['warehouse']))
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

(0, 11, 0)
[Row(CURRENT_WAREHOUSE()='WH_UWM', CURRENT_DATABASE()='UWM_HOUSEHOLDING', CURRENT_SCHEMA()='HOUSEHOLDING')]


In [113]:
snow = session.sql("select * from BONANZA_TABLA").collect()

In [339]:
df = pd.DataFrame(snow, columns = ['CONTACT_ID','FULL_NAME', 'FIRST_NAME', 'MIDDLE', 'LAST_NAME', 'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'ADDRESS_LINE_3', 'CITY', 'STATE', 'ZIP', 'HOUSEHOLD_ID','HOUSEHOLD_IND'])
#snowdf.set_index('CONTACT_ID', inplace = True)
df2 = df
df

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLD_IND
0,1,Claire Walls,Claire,None,Walls,1031189 Metus St,None,Eugene MD 22207,Eugene,MD,22207,99900001,N
1,2,Derek Kane,Derek,None,Kane,1055973 Semper Av,None,Helena OH 24436,Helena,OH,24436,99900002,N
2,3,Stone Mayer,Stone,None,Mayer,1057190 Consectetuer Street,None,West Valley City MA 45162,West Valley City,MA,45162,99900003,N
3,4,Claudia Riddle,Claudia,None,Riddle,1072885 Semper Street,None,Memphis VA 32674,Memphis,VA,32674,99900004,N
4,5,Fuller Parrish,Fuller,None,Parrish,1076016 Augue Ave,None,Cambridge MN 65252,Cambridge,MN,65252,99900005,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,997,George Hardy,George,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y
997,998,Baxter Hardy,Baxter,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y
998,999,Randall Hardy,Randall,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y
999,1000,Jemima Clayton,Jemima,None,Clayton,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y


In [373]:
#'LAST_NAME','ADDRESS_LINE_1','CITY','STATE','ZIP'
features = ['ADDRESS']
X = df.loc[:, features]
y = df.loc[:, ['HOUSEHOLD_ID']]
X_train, X_test, y_train, y_test  = train_test_split(X, y, random_state = 639, test_size = 0.25)

In [341]:
class BytesIntEncoder:

    @staticmethod
    def encode(s: str) -> int:
        if "None" in s:
            s = s.replace("None", " ")
        elif "Ave" in s:
            s = s.replace("Ave", "Avenue")
        elif "Av" in s:
            s = s.replace("Av", "Avenue")
        elif "St" in s and "Str" not in s:
            s = s.replace("St", "Street")
        elif "Rd" in s:
            s = s.replace("Rd", "Road")
        elif "Dr" in s:
            s = s.replace("Dr", "Drive")
        b = s.encode("utf-8")
        return int.from_bytes(b, byteorder='big')

    @staticmethod
    def decode(i: int) -> str:
        b = i.to_bytes(((i.bit_length() + 7) // 8), byteorder='big')
        return b.decode("utf-8")

In [299]:
BytesIntEncoder.encode("1076016 Augue Dr")

'1076016 Augue Drive'

In [325]:
str("NULL")

'NULL'

In [342]:
df2.apply(lambda row : BytesIntEncoder.encode(str(row["ADDRESS_LINE_1"])), axis = 1)
df2.apply(lambda row : BytesIntEncoder.encode(str(row["ADDRESS_LINE_2"])), axis = 1)
df2.apply(lambda row : BytesIntEncoder.encode(str(row["ADDRESS_LINE_3"])), axis = 1)

0                  92326327566006964425372426477707079735
1                  96231036215668337357246262024854582070
2       1404404280402929399863448413363996643999183709...
3               26336562520746358100737953162378618484532
4          1502637041673329913318703127378545275594487090
                              ...                        
996             24312106805635808625004584435480947799602
997             24312106805635808625004584435480947799602
998             24312106805635808625004584435480947799602
999             24312106805635808625004584435480947799602
1000     390562921096222019834816061527777685449714513969
Length: 1001, dtype: object

In [343]:
df

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLD_IND
0,1,Claire Walls,Claire,None,Walls,1031189 Metus St,None,Eugene MD 22207,Eugene,MD,22207,99900001,N
1,2,Derek Kane,Derek,None,Kane,1055973 Semper Av,None,Helena OH 24436,Helena,OH,24436,99900002,N
2,3,Stone Mayer,Stone,None,Mayer,1057190 Consectetuer Street,None,West Valley City MA 45162,West Valley City,MA,45162,99900003,N
3,4,Claudia Riddle,Claudia,None,Riddle,1072885 Semper Street,None,Memphis VA 32674,Memphis,VA,32674,99900004,N
4,5,Fuller Parrish,Fuller,None,Parrish,1076016 Augue Ave,None,Cambridge MN 65252,Cambridge,MN,65252,99900005,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,997,George Hardy,George,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y
997,998,Baxter Hardy,Baxter,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y
998,999,Randall Hardy,Randall,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y
999,1000,Jemima Clayton,Jemima,None,Clayton,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y


In [348]:
df2["ADDRESS"]= df2.apply(lambda row : BytesIntEncoder.encode(str(row["ADDRESS_LINE_1"])), axis = 1).map(str) + ' ' + df2.apply(lambda row : BytesIntEncoder.encode(str(row["ADDRESS_LINE_2"])), axis = 1).map(str) + ' ' + df2.apply(lambda row : BytesIntEncoder.encode(str(row["ADDRESS_LINE_3"])), axis = 1).map(str)
#df2["ADDRESS"] = df2.apply(lambda row : BytesIntEncoder.encode(row["ADDRESS"]), axis = 1)
df2

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLD_IND,ADDRESS
0,1,Claire Walls,Claire,None,Walls,1031189 Metus St,None,Eugene MD 22207,Eugene,MD,22207,99900001,N,2808154430057402045709218859596772227551522584...
1,2,Derek Kane,Derek,None,Kane,1055973 Semper Av,None,Helena OH 24436,Helena,OH,24436,99900002,N,7188879836212993171250920453521849040479479493...
2,3,Stone Mayer,Stone,None,Mayer,1057190 Consectetuer Street,None,West Valley City MA 45162,West Valley City,MA,45162,99900003,N,2023489789301192116775093600924053500109648872...
3,4,Claudia Riddle,Claudia,None,Riddle,1072885 Semper Street,None,Memphis VA 32674,Memphis,VA,32674,99900004,N,7188884270194452822414331118884812347432382794...
4,5,Fuller Parrish,Fuller,None,Parrish,1076016 Augue Ave,None,Cambridge MN 65252,Cambridge,MN,65252,99900005,N,2808157931549301312918827804543180675056453850...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,997,George Hardy,George,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1204453873402152596814666807678691542226555872...
997,998,Baxter Hardy,Baxter,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1204453873402152596814666807678691542226555872...
998,999,Randall Hardy,Randall,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1204453873402152596814666807678691542226555872...
999,1000,Jemima Clayton,Jemima,None,Clayton,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1204453873402152596814666807678691542226555872...


In [352]:
df2['HH_ID'] = df2.groupby('ADDRESS')['CONTACT_ID'].transform('max')

In [192]:
r = "SELECT * FROM df where HOUSEHOLD_ID = 11100003"
r1 = "SELECT * FROM df2 where HH_ID = 550"
sqldf(r1, globals())

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLD_IND,HH_ID
0,546,Octavius Cortez,Octavius,None,Cortez,5229 Vitae St,Apt 182,Fort Worth AL 58712,Fort Worth,AL,58712,11100009,Y,550
1,547,Sean Gardner,Sean,None,Gardner,5229 Vitae St,Apt 182,Fort Worth AL 58712,Fort Worth,AL,58712,11100009,Y,550
2,548,Scarlett Lee Mcgowan,Scarlett,Lee,Mcgowan,5229 Vitae St,Apt 182,Fort Worth AL 58712,Fort Worth,AL,58712,11100009,Y,550
3,549,Stephanie John Mckinney,Stephanie,John,Mckinney,5229 Vitae St,Apt 182,Fort Worth AL 58712,Fort Worth,AL,58712,11100009,Y,550
4,550,Carly Baldwin,Carly,None,Baldwin,5229 Vitae St,Apt 182,Fort Worth AL 58712,Fort Worth,AL,58712,11100009,Y,550


In [357]:
from pandasql import sqldf
q = "SELECT HH_ID, COUNT(HH_ID) AS NO_OF_MEMBERS FROM k group by HH_ID having NO_OF_MEMBERS!= 1"
q1 = "SELECT HOUSEHOLD_ID, COUNT(HOUSEHOLD_ID) AS NO_OF_MEMBERS FROM k1 group by HOUSEHOLD_ID having NO_OF_MEMBERS!= 1"
q2 = "SELECT * FROM df2 WHERE HOUSEHOLD_ID < 11100021"
p = sqldf(q, globals())
p1 = sqldf(q1, globals())
p2 = sqldf(q2, globals())
p2

,CONTACT_ID,FULL_NAME,FIRST_NAME,MIDDLE,LAST_NAME,ADDRESS_LINE_1,ADDRESS_LINE_2,ADDRESS_LINE_3,CITY,STATE,ZIP,HOUSEHOLD_ID,HOUSEHOLD_IND,ADDRESS,HH_ID
0,11,Clementine J Gaines,Clementine,J,Gaines,1190 Porta Street,None,Jacksonville MN 73876,Jacksonville,MN,73876,11100001,Y,16739265087956237780437392575469384459636 32 2...,12
1,12,Althea Gaines,Althea,None,Gaines,1190 Porta Street,None,Jacksonville MN 73876,Jacksonville,MN,73876,11100001,Y,16739265087956237780437392575469384459636 32 2...,12
2,26,Burton Ramirez,Burton,None,Ramirez,1539541 Aenean Rd,None,Vancouver HI 26858,Vancouver,HI,26858,11100002,Y,1097370896325342865573641087801941240652456292...,27
3,27,Gemma Ramirez,Gemma,None,Ramirez,1539541 Aenean Rd,None,Vancouver HI 26858,Vancouver,HI,26858,11100002,Y,1097370896325342865573641087801941240652456292...,27
4,67,Donovan Silva,Donovan,None,Silva,2433648 Ante St,None,Grand Island CO 47088,Grand Island,CO,47088,11100003,Y,1119584521267766605214749984191623429028668788...,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,970,Kaseem Lancaster,Kaseem,None,Lancaster,7764 Arcu St,None,Iowa City AK 99663,Iowa City,AK,99663,11100019,Y,10962429956945386307966218992439605421428 32 1...,970
64,997,George Hardy,George,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1204453873402152596814666807678691542226555872...,999
65,998,Baxter Hardy,Baxter,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1204453873402152596814666807678691542226555872...,999
66,999,Randall Hardy,Randall,None,Hardy,1545 Scelerisque Rd,None,Gresham CO 36762,Gresham,CO,36762,11100020,Y,1204453873402152596814666807678691542226555872...,999


In [371]:
for each in p["HH_ID"]:
    print(each)

12
201
27
289
346
432
482
550
606
663
668
670
71
714
777
83
838
881
936
942
962
970
999


In [69]:
lr = make_pipeline(LinearRegression())
rnc = RadiusNeighborsClassifier(radius=0, outlier_label = "most frequent")
model = make_pipeline(rnc)
model.fit(X_train,y_train.values.ravel())

Pipeline(steps=[('radiusneighborsclassifier',
                 RadiusNeighborsClassifier(outlier_label='most frequent',
                                           radius=0))])

In [ ]:
session.close()